In [1]:
!pip install nltk --quiet

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import nest_asyncio
import sys
from green_mood_tracker.twint_class import TWINT
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import download
from green_mood_tracker.training_data import get_raw_data_notebook

## download and clean data

In [3]:
from green_mood_tracker.data_cleaning import clean
raw_data = get_raw_data_notebook()
raw_data_clean = clean(raw_data,'text')
raw_data_clean.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/henryhall/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/henryhall/.pyenv/versions/3.7.7/envs/lewagon-3.7/lib/python3.7/site-packages/green_mood_tracker/training_data.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sentiment140_final['polarity'] = sentiment140_final.polarity.map({4:2})
/Users/henryhall/.pyenv/versions/3.7.7/envs/lewagon-3.7/lib/python3.7/site-packages/green_mood_tracker/training_data.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

,id,text,polarity,source
77,1680347120,mcdonalds litto si aka cuzin lol cristyyyyy,2.0,sts_gold
78,1835259469,annasaccone love new card would definitely hire,2.0,sts_gold
79,1983068285,supricky wa one enjoyable experience ive youtu...,2.0,sts_gold
80,1559842363,dallas vega goodness httptwitpiccomlzt way suscon,2.0,sts_gold
81,1999078293,jbsfanargentina hey luv pic wa amazing last ch...,2.0,sts_gold


In [4]:
raw_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 831646 entries, 77 to 1599999
Data columns (total 4 columns):
id          831646 non-null object
text        831646 non-null object
polarity    831646 non-null float64
source      831646 non-null object
dtypes: float64(1), object(3)
memory usage: 31.7+ MB


In [27]:
data_sample = raw_data_clean.sample(n=10_000,random_state=0).reset_index()
data_sample.head()

,index,id,text,polarity,source
0,21252,fcc9210884,sat pjs drinking tea watching politics show lo...,2.0,kaggle_sentiment_train
1,1098320,1970470609,tweaked quotone shotquot good could get submit...,2.0,sentiment140
2,1042137,1957161750,thank goodness,2.0,sentiment140
3,1141515,1977185339,gonna tf,2.0,sentiment140
4,908118,1696045747,keithbrooks glad see youre still around,2.0,sentiment140


In [29]:
data_sample['text'] = data_sample.text.apply((lambda x: x.split()))

In [31]:
X.iloc[1]

['tweaked',
 'quotone',
 'shotquot',
 'good',
 'could',
 'get',
 'submitted',
 'story',
 'wish',
 'luck']

In [30]:
from sklearn.model_selection import train_test_split

X = data_sample.text
y = data_sample.polarity

sentence_train, sentence_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

In [8]:
y_train.shape

(7000,)

In [9]:
!pip install tensorflow --quiet

You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(sentence_train)
word_to_id = tk.word_index
id_to_word = {value:key for key,value in word_to_id.items()}
sentences_token_train = tk.texts_to_sequences(sentence_train)
sentences_token_test = tk.texts_to_sequences(sentence_test)
tk.word_counts['film']

14

In [33]:
len(word_to_id)

14776

In [34]:
sentences_token_train[0]

[3806, 3807, 3808, 166, 263, 401]

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def generate_data(X_train,X_test):
    
    # YOUR CODE HERE
    X_train_pad = pad_sequences(X_train,value= -1000, dtype = 'float32',padding='post')
    X__test_pad = pad_sequences(X_test,value= -1000, dtype = 'float32',padding='post')
    
    return X_train_pad,X__test_pad

X_train_pad, X_test_pad = generate_data(sentences_token_train,sentences_token_test)

In [38]:
sentences_token_train[0]

[3806, 3807, 3808, 166, 263, 401]

In [36]:
X_train_pad[0]

array([ 3806.,  3807.,  3808.,   166.,   263.,   401., -1000., -1000.,
       -1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000.,
       -1000., -1000., -1000., -1000., -1000., -1000., -1000., -1000.],
      dtype=float32)

In [15]:
y_train.value_counts()

2.0    6824
1.0     107
0.0      69
Name: polarity, dtype: int64

In [39]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [41]:
y_train.shape

(7000, 3)

In [44]:
vocab_size

14776

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

vocab_size=len(word_to_id)

def init_model(vocab_size):
    
    # YOUR CODE HERE
    model = Sequential()

    model.add(layers.Masking(mask_value=-1000))
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=100))
    model.add(layers.LSTM(units=13, activation='tanh')) 
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(loss= 'CategoricalCrossentropy', 
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

model = init_model(vocab_size)

In [43]:
# model.build()
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [45]:
# YOUR CODE HERE
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5,restore_best_weights=True,monitor='val_accuracy')
history = model.fit(X_train_pad, y_train,
          validation_split= 2/7,
          epochs=50, 
          batch_size=32, 
          verbose=1,
          callbacks = [es])

Epoch 1/50


InvalidArgumentError:  indices[25,6] = -1000 is not in [0, 14776)
	 [[node sequential_2/embedding_2/embedding_lookup (defined at <ipython-input-45-935b164059d4>:10) ]] [Op:__inference_train_function_6577]

Function call stack:
train_function


In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Mean Square Error - Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.show()
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='best')
    plt.show()

In [ ]:
plot_loss(history)

In [ ]:
model.evaluate(X__test_pad,y_test)